# Introduction

**This notebook reads a directory of pdf files, extracts the texts using an OCR package (pytesseract), finetunes an OpenAI Curie model, and runs the model on a subdirectory of pdf files. <br><br>The structure of the notebook goes like this:**
1. [Setup](#Setup)
2. [OCR Text Extraction](#OCR-Text-Extraction)
3. [Training Data Preparation](#Training-Data-Preparation)
4. [Fine Tuning OpenAI's Curie Model](#Fine-Tuning-OpenAI's-Curie-Model)
5. [Validation Data Preparation](#Validation-Data-Preparation)
6. [Running Fine Tuned Model](#Running-Fine-Tuned-Model)

# Setup

In [1]:
from pdf2image import convert_from_bytes
import pytesseract
import os
import pandas as pd
from tqdm import tqdm
import re
import openai
import tiktoken
import random
import json
import fitz
import time
import requests
from requests.packages.urllib3.util import ssl_
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

In [2]:
os.getcwd()

'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\03-CODE\\Base de Reporte'

**Set own root directory.**

In [3]:
root = r'C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE'
os.chdir(root)

**We also define other important directories.**

In [4]:
data_raw = root + r'\01-DATA_PERU\01-DATA_RAW'
data_pro = root + r'\01-DATA_PERU\02-DATA_PROCESSED'
documentation = root + r'\01-DATA_PERU\04-DATA_DOCUMENTATION'

# PDF Document Scraping

In [5]:
OSCE_main_sample = pd.read_stata(data_pro + r'\OSCE_main_sample.dta', convert_dates=True, convert_categoricals=True, index_col=None, convert_missing=False, preserve_dtypes=True, columns=None, order_categoricals=True, chunksize=None, iterator=False, compression='infer', storage_options=None)
OSCE_main_sample

,codigoconvocatoria,n_cod_contrato,urlcontrato,fecha_suscripcion_contrato,year_suscripcion,n_item1,ruc_proveedor1,ruc_destinatario_pago1,n_item2,ruc_proveedor2,...,ruc_destinatario_pago64,n_item65,ruc_proveedor65,ruc_destinatario_pago65,n_item66,ruc_proveedor66,ruc_destinatario_pago66,n_item67,ruc_proveedor67,ruc_destinatario_pago67
0,590320,2008355,http://zonasegura.seace.gob.pe/documentos//srv...,2020-07-13,2020.0,223,20503300525,20503300525,NaN,,...,,NaN,,,NaN,,,NaN,,
1,590320,2006287,http://zonasegura.seace.gob.pe/documentos//srv...,2020-08-11,2020.0,2,20503300525,20503300525,NaN,,...,,NaN,,,NaN,,,NaN,,
2,590320,2003330,http://zonasegura.seace.gob.pe/documentos//srv...,2020-05-28,2020.0,192,20536390201,20536390201,338.0,20536390201,...,,NaN,,,NaN,,,NaN,,
3,591706,2025400,http://zonasegura.seace.gob.pe/documentos//srv...,2020-11-10,2020.0,33,20255361695,20255361695,34.0,20255361695,...,,NaN,,,NaN,,,NaN,,
4,590320,2006532,http://zonasegura.seace.gob.pe/documentos//srv...,2020-06-26,2020.0,295,20503300525,20503300525,386.0,20503300525,...,,NaN,,,NaN,,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,667558,2037565,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2020-12-15,2020.0,5,20332970411,20332970411,NaN,,...,,NaN,,,NaN,,,NaN,,
7496,743189,2080139,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2021-10-21,2021.0,1,20521691191,20521691191,NaN,,...,,NaN,,,NaN,,,NaN,,
7497,839608,2142276,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2022-09-14,2022.0,1,20100096341,20100096341,NaN,,...,,NaN,,,NaN,,,NaN,,
7498,706847,2058868,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2021-05-31,2021.0,1,20600144252,20600144252,NaN,,...,,NaN,,,NaN,,,NaN,,


In [6]:
save_directory = documentation + r'\Main_sample\downloaded_pdfs'

# Create the directory if it doesn't exist
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [7]:
# Override SSL settings
# ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'

# Disable only DH cipher
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'HIGH:!DH:!aNULL'

failed_downloads_df = pd.DataFrame(columns=['n_cod_contrato', 'urlcontrato', 'failed_download'])

# Download and save PDFs
for index, row in tqdm(OSCE_main_sample.iterrows(), total=OSCE_main_sample.shape[0]):
    url = row['urlcontrato']
    contract_code = row['n_cod_contrato']
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=InsecureRequestWarning)
            response = requests.get(url, verify=False)
            if response.status_code == 200:
                # Generate a name for the PDF based on the contract's code
                filename = os.path.join(save_directory, f'pdf_{contract_code}.pdf')
                with open(filename, 'wb') as f:
                    f.write(response.content)
            else:
                failed_downloads_df = pd.concat([failed_downloads_df,pd.DataFrame({'n_cod_contrato': contract_code, 'urlcontrato': url, 'failed_download': 1}, index=[0])], ignore_index=True)
                #failed_downloads_df = failed_downloads_df.append({'Contract Code': contract_code, 'URL': url}, ignore_index=True)
                print(f'Failed to download PDF {contract_code} from {url}. Status code: {response.status_code}')
    except requests.exceptions.SSLError as e:
        print(f"An SSL error occurred: {e} in contract {contract_code} with url: {url}")

  0%|                                                                                | 10/7500 [00:00<02:34, 48.49it/s]

Failed to download PDF 2008355 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/007a7972-8634-4d85-8b0e-6f7426e90c92. Status code: 404
Failed to download PDF 2006287 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/00cd9ad5-3c62-4075-b2a6-e650697288dc. Status code: 404
Failed to download PDF 2003330 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/01750b3f-9674-45f8-a7a4-644ea8e30daf. Status code: 404
Failed to download PDF 2025400 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/01cf55b1-0fe7-4033-9bed-1f9faab2e062. Status code: 404
Failed to download PDF 2006532 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/01e63e7a-ae05-49ee-a61e-f3cec69cd26a. Status code: 404
Failed to download PDF 2017187 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/022e262b-af91-4591-9e4a-237149d4afe8. Status code: 404
Failed to download PDF 2033785 from http://zonasegura.seace.gob.pe/documentos//srv

  0%|▏                                                                               | 22/7500 [00:00<02:22, 52.60it/s]

Failed to download PDF 2016315 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/04192377-860e-4e28-82b3-2b45668c9800. Status code: 404
Failed to download PDF 2011169 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/046379c9-51ea-4cdf-83a4-5934675c50a0. Status code: 404
Failed to download PDF 2014882 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/068d2a89-de53-41ab-a843-9836948ad200. Status code: 404
Failed to download PDF 2011181 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/06e0264c-8ae0-45b1-a181-c902db5bc8b5. Status code: 404
Failed to download PDF 2039549 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/07430560-ece4-424f-9290-16aa85ac530c. Status code: 404
Failed to download PDF 2004538 from http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/07a3f16a-f337-45e3-b595-8ac068c7b401. Status code: 404
Failed to download PDF 2005534 from http://zonasegura.seace.gob.pe/documentos//srv

  0%|▎                                                                               | 35/7500 [00:00<02:12, 56.16it/s]

...

 51%|█████████████████████████████████████▉                                    | 3847/7500 [1:59:46<1:44:42,  1.72s/it]

Failed to download PDF 1300298 from http://zonasegura.seace.gob.pe/documentos/mon\docs\contratos\2020\10486\363773209092020181459.pdf. Status code: 404


 56%|█████████████████████████████████████████▍                                | 4205/7500 [2:13:05<1:15:42,  1.38s/it]

Failed to download PDF 1291636 from http://zonasegura.seace.gob.pe/documentos/mon\docs\contratos\2020\22\364466711082020163110.pdf. Status code: 404


100%|████████████████████████████████████████████████████████████████████████████| 7500/7500 [3:17:55<00:00,  1.58s/it]


In [8]:
failed_downloads_df.drop('urlcontrato',
  axis='columns', inplace=True)

# OCR Text Extraction

**OCR extraction with Tesseract in Windows requires to have the program installed from this [link](https://github.com/UB-Mannheim/tesseract/wiki).<br> After installing, we need to specify the location of the exe file as below.**

In [9]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

**We first define the directory where the pdfs are located.**

In [10]:
pdfs = documentation + r'\Main_sample\downloaded_pdfs'

**We span through the files of the base pdf directory, and its subdirectories, storing the filepath of each document.**

In [11]:
filenames = []

for base, dirs, files in os.walk(pdfs):
    for filename in files:
        if (filename.lower().endswith('.pdf')):
            filenames.append(os.path.join(base, filename))

random.choices(filenames, k=5)

['C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Main_sample\\downloaded_pdfs\\pdf_1280466.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Main_sample\\downloaded_pdfs\\pdf_2151431.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Main_sample\\downloaded_pdfs\\pdf_2116665.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Main_sample\\downloaded_pdfs\\pdf_2133613.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Main_sample\\downloaded_pdfs\\pdf_2112068.pdf']

**We open each path and extract the text using the fitz package (a simple pdf reader). <br>Then we identify if the pdf was a scanned document (an image without *"highlight-able"* text) by setting a length threshold. <br>Only if the read pdf's text has a length below the threshold, we proceed by re-extracting the text using the Tesseract OCR package. <br>Each text is stored in a dataframe containing its filename, text, extraction type and file id.**

In [12]:
# Create a dataframe to store the texts of each PDF
pdf_texts_df = pd.DataFrame(columns = ['filename', 'text', 'extraction_type'])
broken_pdfs_df = pd.DataFrame(columns=['n_cod_contrato', 'broken_pdf'])

# Loop through every file in the directory
for filename in tqdm(filenames):
    if filename.lower().endswith('.pdf'):
        try:
        
            # Read the text directly from the PDF file
            reader = fitz.open(filename)
            pdf_text = ''

            for page in reader:
                pdf_text+=page.get_text()+' '
            
            pdf_texts_df.loc[filenames.index(filename), 'extraction_type'] = 'PDF_Reader'
            
            if len(re.sub(r'[^a-zA-Z]', '', pdf_text))<1000:
            
                # Open the PDF file
                with open(filename, 'rb') as file:
                    pdf_bytes = file.read()

                # Convert the PDF to images
                images = convert_from_bytes(pdf_bytes)

                # Use OCR to extract text from each image/page
                pdf_text = ''
                for i, image in enumerate(images):
                    text = pytesseract.image_to_string(image)
                    pdf_text+=text+' '
            
                pdf_texts_df.loc[filenames.index(filename), 'extraction_type'] = 'OCR'

            # Clean the extracted text

            clean_text = re.sub('\$+', ' ', pdf_text)  # Replace multiple \$ with a space
            clean_text = re.sub('\n+', ' ', clean_text)  # Replace multiple newlines with one space
            clean_text = re.sub('\.+', '.', clean_text)  # Replace multiple . with one space
            clean_text = re.sub('\,+', ',', clean_text)  # Replace multiple newlines with one space
            clean_text = clean_text.replace(';', ' ')  # Replace semicolons with spaces
            clean_text = re.sub(' +', ' ', clean_text)  # Replace multiple spaces with one
            clean_text = re.sub(r'[^a-zA-ZÀ-ÿ0-9 \,\.\/\:]', '', clean_text)
        
            # Store the joined text in the dataframe, using the filename (without .pdf) as the key
            pdf_texts_df.loc[filenames.index(filename), 'filename'] = filename[:-4]
            pdf_texts_df.loc[filenames.index(filename), 'text'] = clean_text
            
        except Exception as e:
            error_message = str(e)
            if 'cannot open broken document' in error_message:  # Replace 'FileDataError' with the actual error message you expect
                print(f"A FileDataError occurred: {e} in {filename}")
                broken_pdfs_df = pd.concat([broken_pdfs_df, pd.DataFrame({'n_cod_contrato': [filename.replace(pdfs+'\\pdf_', '').replace('.pdf','')], 'broken_pdf': 1}, index=[0])], ignore_index=True)
                #broken_pdfs_df = pd.concat([broken_pdfs_df, pd.DataFrame({'Contract Code': filename.str.replace(pdfs+'\\pdf_', '', regex=False)}, index=[0])], ignore_index=True)
            else:
                print(f"An unspecified error occurred: {e} in {filename}")
                broken_pdfs_df = pd.concat([broken_pdfs_df, pd.DataFrame({'n_cod_contrato': [filename.replace(pdfs+'\\pdf_', '').replace('.pdf','')], 'broken_pdf': 1}, index=[0])], ignore_index=True)


  0%|                                                                                         | 0/6887 [00:00<?, ?it/s]

A FileDataError occurred: cannot open broken document in C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE\01-DATA_PERU\04-DATA_DOCUMENTATION\Main_sample\downloaded_pdfs\pdf_1146041.pdf


  0%|                                                                              | 6/6887 [01:12<30:06:44, 15.75s/it]

A FileDataError occurred: cannot open broken document in C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE\01-DATA_PERU\04-DATA_DOCUMENTATION\Main_sample\downloaded_pdfs\pdf_1149185.pdf


  1%|▌                                                                            | 50/6887 [12:17<22:08:07, 11.66s/it]

...

A FileDataError occurred: cannot open broken document in C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE\01-DATA_PERU\04-DATA_DOCUMENTATION\Main_sample\downloaded_pdfs\pdf_1174730.pdf


  4%|██▊                                                                       | 264/6887 [1:38:16<61:20:07, 33.34s/it]

A FileDataError occurred: cannot open broken document in C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE\01-DATA_PERU\04-DATA_DOCUMENTATION\Main_sample\downloaded_pdfs\pdf_1177078.pdf


  5%|███▊                                                                      | 353/6887 [2:21:34<43:40:38, 24.06s/it]


KeyboardInterrupt: 

In [ ]:
pdf_texts_df['n_cod_contrato'] = pdf_texts_df['filename'].str.replace(pdfs+'\\pdf_', '', regex=False)
pdf_texts_df.drop('filename', axis='columns', inplace=True)

In [ ]:
main_analysis_dfs = data_pro + r'\main_analysis_dfs'

# Create the directory if it doesn't exist
if not os.path.exists(main_analysis_dfs):
    os.makedirs(main_analysis_dfs)

In [ ]:
display(failed_downloads_df)
display(broken_pdfs_df)
display(pdf_texts_df)

In [ ]:
pdf_texts_df.to_excel(main_analysis_dfs + r'\pdf_texts.xlsx', index = False)
failed_downloads_df.to_excel(main_analysis_dfs + r'\failed_downloads.xlsx', index = False)
broken_pdfs_df.to_excel(main_analysis_dfs + r'\broken_pdfs.xlsx', index = False)

# Preparing Data for Unit Price Extraction

In [17]:
extraction_df = pd.read_excel(data_pro + r'\main_analysis_dfs\pdf_texts.xlsx')

In [24]:
extraction_df

,text,extraction_type,n_cod_contrato
0,ee rey ela par ee res contrato n0012018hh se...,OCR,1147865.0
1,4 see . hospital peru ministerio. de salud ca...,OCR,1148734.0
2,es ieee a x : eae . be ap pe peru ministerio...,OCR,1149147.0
3,fox hospital . 7m peru ministerio de salud...,OCR,1149733.0
4,ey : ea we : hospital ee peru ministerio:de...,OCR,1149901.0
...,...,...,...
3935,nes peru ministeriode salud 8 licitacion publi...,OCR,2168959.0
3936,"patricia , piedra z on errod 0 i minsterio s...",OCR,2169593.0
3937,"see vi sli eea aes cit , iceministerio de rede...",OCR,2170324.0
3938,marina de guerra del peru nt direcion ejecutiv...,OCR,2170926.0


In [30]:
extraction_df['text'].fillna('NA', inplace=True)
extraction_df['text'] = extraction_df['text'].str.lower()
extraction_df['text'] = extraction_df['text'].apply(lambda x: re.sub('\s+', ' ', x).strip())
extraction_df['text'] = extraction_df['text'].apply(lambda x: re.sub(r'(\.|\,)\1+', r'\1', x).strip())
extraction_df.drop('extraction_type', axis='columns', inplace=True)

In [31]:
# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    """Yield successive n-sized chunks from text."""
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.9 * n and 1.1 * n tokens
        j = min(i + int(1.1 * n), len(tokens))
        while j > i + int(0.9 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.9 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j

In [32]:
# Initialise tokenizer
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

prompt=[]

for i in tqdm(range(len(extraction_df['text']))):
    chunks = create_chunks(extraction_df.loc[i,'text'], 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    if len(text_chunks)>=2:
        prompt.append(' '.join(text_chunks[:1]+text_chunks[-1:]))
    else:
        prompt.append(' '.join(text_chunks))
extraction_df['text'] = prompt

100%|██████████████████████████████████████████████████████████████████████████████| 3940/3940 [01:31<00:00, 43.11it/s]


In [33]:
extraction_df['text'] = extraction_df['text'] + '---->'

In [34]:
os.environ['OPENAI_API_KEY'] = ""

In [35]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [36]:
# Iterate through each row in DataFrame
for index, row in tqdm(extraction_df.iterrows(), total=extraction_df.shape[0]):
    user_content = row['text']
    time.sleep(2) 
    
    try:
        # Call OpenAI API
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message

    except openai.error.RateLimitError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"Rate limit error. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message
        
    except openai.error.ServiceUnavailableError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"Service Unavailable error. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message

    except openai.error.APIError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"API error occurred. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message

    except OSError as e:
        retry_time = 1  # Adjust the retry time as needed
        print(f"Connection error occurred: {e}. Retrying in {retry_time} seconds...")      
        time.sleep(retry_time)
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message
        
    except requests.Timeout as e:
        retry_time = 1  # Adjust the retry time as needed
        print(f"Timeout error occurred: {e}. Retrying in {retry_time} seconds...")      
        time.sleep(retry_time)
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message
        
    except Exception as e:
        retry_time = 1
        print(f"An unexpected error occurred: {e}. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message
        
# Print the updated DataFrame to check if it worked
extraction_df

 68%|███████████████████████████████████████████████████▊                        | 2683/3940 [2:05:35<55:01,  2.63s/it]

API error occurred. Retrying in 1 seconds...


100%|████████████████████████████████████████████████████████████████████████████| 3940/3940 [3:02:26<00:00,  2.78s/it]


,text,n_cod_contrato,gpt_unit_prices
0,ee rey ela par ee res contrato n0012018hh serv...,1147865.0,8.0; 16.0; 16.0; 8.0; 16.2; 7.5; 7.5; 16.2; 7...
1,4 see . hospital peru ministerio. de salud cay...,1148734.0,0.28; 0.25; 0.22; 0.12; 0.12; 0.35; 0.35; 0.3...
2,es ieee a x : eae . be ap pe peru ministerio i...,1149147.0,5276.66; 3650.0 \n\n###\n\n
3,fox hospital . 7m peru ministerio de salud cay...,1149733.0,4.667 \n\n###\n\n
4,ey : ea we : hospital ee peru ministerio:de sa...,1149901.0,0.3; 4.0; 1.0; 1.0; 1.0; 3.35; 2.3; 2.5; 1.0;...
...,...,...,...
3935,nes peru ministeriode salud 8 licitacion publi...,2168959.0,\n\n###\n\n
3936,"patricia , piedra z on errod 0 i minsterio ss ...",2169593.0,6.726; 6.785 \n\n###\n\n
3937,"see vi sli eea aes cit , iceministerio de rede...",2170324.0,135.0; 48.0; 67.0; 75.0; 67.0 \n\n###\n\n
3938,marina de guerra del peru nt direcion ejecutiv...,2170926.0,0.08; 2.42; 0.38; 2.2; 0.84 \n\n###\n\n


In [37]:
extraction_df.to_excel(main_analysis_dfs + r'\extraction_df.xlsx', index = False)